# Using CAVE with AutoPyTorch

AutoPyTorch provides a framework for automated neural-network-configuration. Currently it supports [BOHB](https://github.com/automl/HpBandSter) for hyperparameter search.
CAVE integrates with AutoPyTorch, providing further insights and visualizations.
This notebook provides an exemplary pipeline for using CAVE on / with AutoPyTorch.

We will generate some AutoPyTorch-Output.
You can use your own AutoPyTorch-routine here, we will use the openml-tasks, inspired by [AutoPyTorch's tutorial notebook](https://github.com/automl/Auto-PyTorch/blob/master/examples/basics/Auto-PyTorch%20Tutorial.ipynb).

Note: This example adapts with the refactor of the APT project.
Since logging is not yet finally implemented in the APT project, this example is not necessarily fully executable...
However, feel free to open issues on errors you encounter in the [issue tracker](https://github.com/automl/CAVE/issues).

In [1]:
# Remove the old example output
import os
import logging
import tempfile
import shutil
log_dir = "logs/apt-cave-notebook/"
rerun_apt = False

logging.basicConfig(level=logging.DEBUG)

In [2]:
from autoPyTorch import AutoNetClassification
import os as os
import openml
import json
from ConfigSpace.read_and_write import json as pcs_json
# Logging
from autoPyTorch.components.metrics.additional_logs import *
from autoPyTorch.pipeline.nodes import LogFunctionsSelector

if rerun_apt:
    # Remove old results
    if os.path.exists(log_dir):
        archive_path = shutil.make_archive(os.path.join(tempfile.mkdtemp(), '.OLD'), 'zip', log_dir)
        shutil.rmtree(log_dir)
        os.makedirs(log_dir)
        shutil.move(archive_path, log_dir)
    else:
        os.makedirs(log_dir)


    task = openml.tasks.get_task(task_id=31)

    X, y = task.get_X_and_y()
    ind_train, ind_test = task.get_train_test_split_indices()
    X_train, Y_train = X[ind_train], y[ind_train]
    X_test, Y_test = X[ind_test], y[ind_test]

    autopytorch = AutoNetClassification(config_preset="medium_cs",
                                        result_logger_dir=log_dir,
                                        log_every_n_datapoints=10,
                                        use_tensorboard_logger=True,
                                        additional_logs=[test_result.__name__,
                                                         test_cross_entropy.__name__,
                                                         test_balanced_accuracy.__name__],
                                       )

    # Get data from the openml task "Supervised Classification on credit-g (https://www.openml.org/t/31)"
    task = openml.tasks.get_task(task_id=31)
    X, y = task.get_X_and_y()
    ind_train, ind_test = task.get_train_test_split_indices()
    X_train, Y_train = X[ind_train], y[ind_train]
    X_test, Y_test = X[ind_test], y[ind_test]
    
    
    # Equip autopytorch with additional logs
    gl = GradientLogger()
    lw_gl = LayerWiseGradientLogger()
    additional_logs = [gradient_max(gl), gradient_mean(gl), gradient_median(gl), gradient_std(gl),
                       gradient_q10(gl), gradient_q25(gl), gradient_q75(gl), gradient_q90(gl),
                       layer_wise_gradient_max(lw_gl), layer_wise_gradient_mean(lw_gl),
                       layer_wise_gradient_median(lw_gl), layer_wise_gradient_std(lw_gl),
                       layer_wise_gradient_q10(lw_gl), layer_wise_gradient_q25(lw_gl),
                       layer_wise_gradient_q75(lw_gl), layer_wise_gradient_q90(lw_gl),
                       gradient_norm()]

    for additional_log in additional_logs:
        autopytorch.pipeline[LogFunctionsSelector.get_name()].add_log_function(name=type(additional_log).__name__,
                                                                               log_function=additional_log)

        #sampling_space["additional_logs"].append(type(additional_log).__name__)

    autopytorch.pipeline[LogFunctionsSelector.get_name()].add_log_function(name=test_result.__name__, 
                                                                           log_function=test_result(autopytorch, X[ind_test], y[ind_test]))
    autopytorch.pipeline[LogFunctionsSelector.get_name()].add_log_function(name=test_cross_entropy.__name__,
                                                                           log_function=test_cross_entropy(autopytorch, X[ind_test], y[ind_test]))
    autopytorch.pipeline[LogFunctionsSelector.get_name()].add_log_function(name=test_balanced_accuracy.__name__,
                                                                           log_function=test_balanced_accuracy(autopytorch, X[ind_test], y[ind_test]))

    # Fit to find an incumbent configuration with BOHB
    results_fit = autopytorch.fit(X_train=X_train,
                                  Y_train=Y_train,
                                  validation_split=0.3,
                                  max_runtime=750,
                                  min_budget=10,
                                  max_budget=50,
                                  refit=True,
                                 )
    autopytorch.refit_all_incumbents(X_train, Y_train)

/home/shuki/niki/virtualenvs/CAVE_dev/lib/python3.8/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.metrics.classification module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)


Note: APT is supposed to automatically log the results to the output directory. Until then, do in manually:

In [3]:
if rerun_apt:
    # Save fit results as json
    with open(os.path.join(log_dir, "results_fit.json"), "w") as f:
        json.dump(results_fit, f, indent=2)
    
    # Also necessary information (can be migrated either to CAVE or (preferably) to autopytorch)
    with open(os.path.join(log_dir, 'configspace.json'), 'w') as f:
        f.write(pcs_json.write(autopytorch.get_hyperparameter_search_space(X_train=X_train,
                                                                       Y_train=Y_train)))
    with open(os.path.join(log_dir, 'autonet_config.json'), 'w') as f:
        json.dump(autopytorch.get_current_autonet_config(), f, indent=2)

Next, spin up CAVE pass along the output directory.

In [4]:
from cave.cavefacade import CAVE

cave_output_dir = "cave_output"

cave = CAVE([log_dir],        # List of folders holding results
            cave_output_dir,  # Output directory
            ['.'],            # Target Algorithm Directory (only relevant for SMAC)
            file_format="APT",
            verbose="DEBUG")

/home/shuki/niki/virtualenvs/CAVE_dev/lib/python3.8/site-packages/pyparsing.py:3174: FutureWarning: Possible set intersection at position 3
  self.re = re.compile(self.reString)
DEBUG:matplotlib:CONFIGDIR=/home/shuki/.config/matplotlib
DEBUG:matplotlib:(private) matplotlib data path: /home/shuki/niki/virtualenvs/CAVE_dev/lib/python3.8/site-packages/matplotlib/mpl-data
DEBUG:matplotlib:matplotlib data path: /home/shuki/niki/virtualenvs/CAVE_dev/lib/python3.8/site-packages/matplotlib/mpl-data
DEBUG:matplotlib:loaded rc file /home/shuki/niki/virtualenvs/CAVE_dev/lib/python3.8/site-packages/matplotlib/mpl-data/matplotlibrc
DEBUG:matplotlib:matplotlib version 3.2.1
DEBUG:matplotlib:interactive is False
DEBUG:matplotlib:platform is linux
DEBUG:matplotlib:loaded modules: ['sys', 'builtins', '_frozen_importlib', '_imp', '_warnings', '_frozen_importlib_external', '_io', 'marshal', 'posix', '_thread', '_weakref', 'time', 'zipimport', '_codecs', 'codecs', 'encodings.aliases', 'encodings', 'encodi

DEBUG:matplotlib:CACHEDIR=/home/shuki/.cache/matplotlib
DEBUG:matplotlib.font_manager:Using fontManager instance from /home/shuki/.cache/matplotlib/fontlist-v310.json
DEBUG:matplotlib.pyplot:Loaded backend agg version unknown.
DEBUG:matplotlib.pyplot:Loaded backend agg version unknown.
/home/shuki/niki/virtualenvs/CAVE_dev/lib/python3.8/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.metrics.regression module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
/home/shuki/niki/virtualenvs/CAVE_dev/lib/python3.8/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.manifold.mds module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported

In [5]:
cave.apt_overview()

,apt-cave-notebook
embeddings,[none]
lr_scheduler,"[cosine_annealing, plateau]"
networks,[shapedresnet]
over_sampling_methods,[smote]
preprocessors,"[none, truncated_svd, power_transformer]"
target_size_strategies,"[none, upsample, median]"
result_logger_dir,logs/apt-cave-notebook/
log_every_n_datapoints,10
use_tensorboard_logger,True
additional_logs,"[test_result, test_cross_entropy, test_balanced_accuracy]"


In [6]:
cave.compare_default_incumbent()

,Default,Incumbent
--------------- Changed parameters: ---------------,-----,-----
CreateDataLoader:batch_size,126,248
LearningrateSchedulerSelector:lr_scheduler,plateau,cosine_annealing
NormalizationStrategySelector:normalization_strategy,standardize,maxabs
PreprocessorSelector:preprocessor,power_transformer,truncated_svd
ResamplingStrategySelector:target_size_strategy,median,none
TrainNode:batch_loss_computation_technique,standard,mixup
LearningrateSchedulerSelector:cosine_annealing:T_max,inactive,202
LearningrateSchedulerSelector:cosine_annealing:eta_min,inactive,1e-08
LearningrateSchedulerSelector:plateau:factor,0.275,inactive


Other analyzers also run on the APT-data:

In [7]:
cave.apt_tensorboard()

cave_output/converted_input_data/apt-cave-notebook
